In [1]:
from sqlite3 import connect
import pandas as pd

conn = connect("machine2.db")

In [ ]:
df = pd.read_sql_query("SELECT * FROM depression", conn);

q_mappings = {
  'affect': [1, 4, 5, 10, 11],
  'cognitive': [12, 13, 15], 
  'somatic': [16, 17, 18, 19, 20, 21],
  'outlook': [2, 3, 6, 7, 8, 9]
}
categories = [None] * 22
for category, questions in q_mappings.items():
  for question in questions:
    categories[question] = category

def check_complete(series: pd.Series):
  mask = (series >= 0) & (series <= 3)
  n_values = mask.sum()
  if n_values == len(series):
    return 1
  if n_values == 0:
    return -1
  return 0

df['category'] = df['instanceId'].apply(lambda x: categories[x])
df['agreement_level'] = df['agreement_level'].apply(lambda x: x if type(x) is int else int.from_bytes(x) if type(x) is bytes else -1)
df['complete'] = df.groupby('participantId')['agreement_level'].transform(check_complete)


df.head()

,participantId,trialNumber,taskId,instanceId,agreement_level,reaction_time,category,complete
0,97f4_7385706431,89,2,11,-1,None,affect,-1
1,97f4_7385706431,105,2,2,-1,None,outlook,-1
2,97f4_7385706431,111,2,5,-1,None,affect,-1
3,97f4_7385706431,147,2,8,-1,None,outlook,-1
4,97f4_7385706431,153,2,6,-1,None,outlook,-1


In [3]:
aggregate = lambda s: (lambda x: x if pd.notna(x) else 0)(s[(s >= 0) & (s <= 3)].mean())
agg_df = df.pivot_table(
    index='participantId',
    columns='category',
    values='agreement_level',
    aggfunc=aggregate,
)

agg_df.columns.name = None

agg_df['Overall'] = df.groupby('participantId')['agreement_level'].apply(aggregate)
complete_series = df.groupby('participantId')['complete'].first()
agg_df['complete'] = agg_df.index.map(complete_series.to_dict())
agg_df.head()

,affect,cognitive,outlook,somatic,Overall,complete
participantId,,,,,,
97f4_7385706431,0.0,0.000000,0.000000,0.000000,0.000000,-1
a265_7382527621,1.2,0.666667,0.833333,1.000000,0.952381,1
a295_7382607093,0.0,0.000000,0.000000,0.000000,0.000000,-1
a295_7382607228,0.6,0.333333,1.000000,0.333333,0.619048,1
a3a7_7375337601,0.4,0.666667,0.166667,0.833333,0.476190,1


In [4]:
agg_df.to_csv('depression.csv')